# 5.3 Summarizing and Computing Descriptive Statistics（汇总和描述性统计）

pandas有很多数学和统计方法。大部分可以归类为降维或汇总统计，这些方法是用来从series中提取单个值（比如sum或mean）。还有一些方法来处理缺失值：

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


使用sum的话，会返回一个series:

In [4]:
df.sum()

one    9.25
two   -5.80
dtype: float64

使用`axis='columns'` or `axis=1`，计算列之间的和：

In [5]:
df.sum(axis='columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

计算的时候，NA（即缺失值）会被除外，除非整个切片全是NA。我们可以用skipna来跳过计算NA：

In [6]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

一些reduction方法：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/n95fy.png)

一些方法，比如idxmin和idxmax，能返回间接的统计值，比如index value：

In [8]:
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [7]:
df.idxmax()

one    b
two    d
dtype: object

还能计算累加值：

In [9]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


另一种类型既不是降维，也不是累加。describe能一下子产生多维汇总数据：

In [10]:
df.describe()

/Users/xu/anaconda/envs/py35/lib/python3.5/site-packages/numpy/lib/function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,NaN,NaN
50%,NaN,NaN
75%,NaN,NaN
max,7.100000,-1.300000


对于非数值性的数据，describe能产生另一种汇总统计：

In [11]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj

0     a
1     a
2     b
3     c
4     a
5     a
6     b
7     c
8     a
9     a
10    b
11    c
12    a
13    a
14    b
15    c
dtype: object

In [12]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

下表是一些描述和汇总统计数据：
![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/wygi1.png)

# 1 Correlation and Covariance (相关性和协方差)

假设DataFrame时股价和股票数量。这些数据取自yahoo finace，用padas-datareader包能加载。如果没有的话，用conda或pip来下载这个包：

In [ ]:
conda install pandas-datareader

In [14]:
import pandas_datareader.data as web

In [ ]:
all_data = {ticker: web.get_data_yahoo(ticker)
            for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}

price = pd.DataFrame({ticker: data['Adj Close']
                      for ticker, data in all_data.items()})

volumn = pd.DataFrame({ticker: data['Volumn']
                       for ticker, data in all_data.items()})

上面的代码无法直接从yahoo上爬取数据，因为yahoo被verizon收购后，好像是不能用了。于是这里我们直接从下好的数据包里加载。

In [17]:
ls ../examples/

5.1 Introduction to pandas Data Structures（pandas的数据结构）.ipynb
5.2 Essential Functionality（主要功能）.ipynb
5.3 Summarizing and Computing Descriptive Statistics（总结和描述性统计）.ipynb


In [18]:
price = pd.read_pickle('../examples/yahoo_price.pkl')
volume = pd.read_pickle('../examples/yahoo_volume.pkl')

In [20]:
price.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571


In [21]:
volume.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,123432400,3927000,6155300,38409100
2010-01-05,150476200,6031900,6841400,49749600
2010-01-06,138040000,7987100,5605300,58182400
2010-01-07,119282800,12876600,5840600,50559700
2010-01-08,111902700,9483900,4197200,51197400


> pct_change(): 这个函数用来计算同colnums两个相邻的数字之间的变化率

现在我们计算一下价格百分比的变化：

In [23]:
returns = price.pct_change()
returns.tail()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2016-10-17,-0.000680,0.001837,0.002072,-0.003483
2016-10-18,-0.000681,0.019616,-0.026168,0.007690
2016-10-19,-0.002979,0.007846,0.003583,-0.002255
2016-10-20,-0.000512,-0.005652,0.001719,-0.004867
2016-10-21,-0.003930,0.003011,-0.012474,0.042096


series的corr方法计算两个，重合的，非NA的，通过index排列好的series。cov计算方差：

In [24]:
returns['MSFT'].corr(returns['IBM'])

0.4997636114415116

In [26]:
returns['MSFT'].cov(returns['IBM'])

8.8706554797035489e-05

因为MSFT是一个有效的python属性，我们可以通过更简洁的方式来选中columns：

In [27]:
returns.MSFT.corr(returns.IBM)

0.4997636114415116

dataframe的corr和cov方法，能返回一个完整的相似性或方差矩阵：

In [28]:
returns.corr()

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.407919,0.386817,0.389695
GOOG,0.407919,1.000000,0.405099,0.465919
IBM,0.386817,0.405099,1.000000,0.499764
MSFT,0.389695,0.465919,0.499764,1.000000


In [29]:
returns.cov()

,AAPL,GOOG,IBM,MSFT
AAPL,0.000277,0.000107,0.000078,0.000095
GOOG,0.000107,0.000251,0.000078,0.000108
IBM,0.000078,0.000078,0.000146,0.000089
MSFT,0.000095,0.000108,0.000089,0.000215


用Dataframe的corrwith方法，我们可以计算dataframe中不同columns之间，或row之间的相似性。传递一个series：

In [30]:
returns.corrwith(returns.IBM)

AAPL    0.386817
GOOG    0.405099
IBM     1.000000
MSFT    0.499764
dtype: float64

传入一个dataframe能计算匹配的column names质监局的相似性。这里我计算vooumn中百分比变化的相似性：

In [32]:
returns.corrwith(volume)

AAPL   -0.075565
GOOG   -0.007067
IBM    -0.204849
MSFT   -0.092950
dtype: float64

传入axis='columns'能做到row-by-row计算。在correlation被计算之前，所有的数据会根据label先对齐。

# 2 Unique Values, Value Counts, and Membership（唯一值，值计数，会员）

这里介绍另一种从一维series中提取信息的方法：

In [33]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

第一个函数时unique，能告诉我们series里unique values有哪些：

In [34]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

返回的unique values不是有序的，但我们可以排序，uniques.sort()。相对的，value_counts能计算series中值出现的频率：

In [35]:
obj.value_counts()

a    3
c    3
b    2
d    1
dtype: int64

返回的结果是按降序处理的。vaule_counts也是pandas中的方法，能用在任何array或sequence上：

In [36]:
pd.value_counts(obj.values, sort=False)

d    1
c    3
b    2
a    3
dtype: int64

isin 能实现一个向量化的集合成员关系检查，能用于过滤数据集，检查一个子集，是否在series的values中，或在dataframe的column中：

In [37]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [38]:
mask = obj.isin(['b', 'c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [39]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

与isin相对的另一个方法是Index.get_indexer，能返回一个index array，告诉我们有重复值的values(to_match)，在非重复的values(unique_vals)中对应的索引值：

In [40]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])

In [41]:
pd.Index(unique_vals).get_indexer(to_match)

array([0, 2, 1, 1, 0, 2])

Unique, value counts, and set membership methods：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/0v120.png)

在某些情况下，你可能想要计算一下dataframe中多个column的柱状图：

In [42]:
data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                     'Qu2': [2, 3, 1, 2, 3],
                     'Qu3': [1, 5, 2, 4, 4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


把padas.value_counts传递给dataframe的apply函数：

In [44]:
result = data.apply(pd.value_counts)
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,NaN,2.0,1.0
3,2.0,2.0,NaN
4,2.0,NaN,2.0
5,NaN,NaN,1.0


每一行的laebls(即1，2，3，4，5)其实就是整个data里出现过的值，从1到5。而对应的每个方框里的值，则是表示该值在当前列中出现的次数。比如，(2, Qu1)的值是Nan，说明2这个数字没有在Qu1这一列出现过。(2, Qu2)的值是2，说明2这个数字在Qu2这一列出现过2次。(2, Qu3)的值是1，说明2这个数字在Qu3这一列出现过1次。